In [2]:
pip install git+https://github.com/SengerM/landaupy

  Cloning https://github.com/SengerM/landaupy to /tmp/pip-req-build-zhd888t2
  Running command git clone --filter=blob:none -q https://github.com/SengerM/landaupy /tmp/pip-req-build-zhd888t2
  Resolved https://github.com/SengerM/landaupy to commit 5142316dd6c1f6f4ad19c6d6b5f174fe0a4b7959
  Preparing metadata (setup.py) ... done
  Created wheel for landaupy: filename=landaupy-0.1-py3-none-any.whl size=10645 sha256=196393042ad0367ab69eeca98d37983b1b0240a14796e4b2849345637f9e5a92
  Stored in directory: /tmp/pip-ephem-wheel-cache-r15ybcx4/wheels/35/24/cd/016184b333ff9e2b0f635dfa82e89401a47fdc83d4a836bbbc
Successfully built landaupy
Note: you may need to restart the kernel to use updated packages.


In [10]:
"""
Pixel Firing Threshold
"""
c=3*10**8 #Speed of light in m.s^{-1}
me=5.11*10**-4 #Mass of the electron in GeV/c²

#we use the library git+https://github.com/SengerM/landaupy that is included in requirement.txt (virtual environment)
import landaupy
from landaupy import landau
import matplotlib.pyplot as plt
import numpy as np

#Initial position:

z0=0.1
y0=0.1

#Initial momentum in GeV/c:

p_z0=-3
p_y0=0

#Initial velocities in c:

v_z0=p_z0/me
v_y0=p_y0/me

#constants

j=0.200 #density effect correction
K=0.3*6.02*(10**23)*10**6 #A MODIFIER CF PDG
th=14e-4 #detector thickness (in m)
rho=2.2 #density of detector (in g/cm3)
x=rho*th #detector thickness (in g/cm2)
v=np.sqrt(v_z0**2+v_y0**2)
beta=(v/c)**2
g=np.sqrt(1-beta**2) #inverse Lorentz coefficient
threshold=100 #environ 100e-
ratio_ZA=0.49930
ksi=(K/2)*ratio_ZA*(x/(beta**2))
w=4.018*ksi #width of Landau distribution
Wi=3.6 #minimum energy required to create an electron-hole pair for Si (in eV)

#parameters
def most_probable_value(Z): 
    if Z<13:
        I=12+7/Z #average excitation energy
    else:
        I=9.76+58.8*(Z**(-1.19))
    delta_p=ksi*(np.log((2*me*10**(9)*((beta)/g)**2)/I)+np.log(ksi/I)+j-beta**2) #most probable value
    return(delta_p)
    
delta_p_si=most_probable_value(14) #Z=14 for silicium

#we want nb of electron hole pair = energy deposited/Wi
#creates 100 random values from the Landau distribution

X=np.linspace(delta_p_si-1E8,delta_p_si+1E8,100000)
pdf = landau.pdf(X, delta_p_si, w)

samples = landau.sample(x_mpv=delta_p_si, xi=w, n_samples=9999)
#x_mpv is the most probable energy of Landau distribution
#xi is the width
#Parameters:

from random import gauss

q=-0.3 #for a particle with a charge 1.6*10**-19 C
B=0.001 #Magnetic field in T
L=0.01 #length of the magnetic zone, in m
d=0.005 #length of the dectectors

#multiple scattering
def angulardistribution(E):
    "take in argument a list of energy in Gev, and return the list of scattering angles, following gauss distribution"
    scatter_angles=[]
    for i in E:
        p=np.sqrt(2*me*i)
        v=p/me
        theta=(13.6/(p*v))*q*np.sqrt(x/(27.05*10**4))*(1+0.038*np.log(x/(27.05*10**4)))
        angle=gauss(0,theta)
        scatter_angles.append(angle)
    return(scatter_angles)

theta_scattering=angulardistribution(samples)
sample_after_scattering=np.cos(theta_scattering)*samples #we calculate the modified energy after multiple scattering in the previous detector

In [12]:
#back to pixel firing threshold
def list_paires(sample,seuil):
    list=[]
    for i in range(0,len(sample)):
        list.append(sample[i]//seuil)
    return(list)
  
#the function takes as argument a random list of energies according to the landau distribution
#and the electron hole pair creation threshold, it returns a list containing the number of times
#that we have the threshold in each of the energy values in the list
#(Euclidean division of the energy deposited by the threshold energy)

def compte_pixel(list_paires,threshold):
    n=0
    for i in range(0,len(list_paires)):
        if list_paires[i]>=threshold:
            n+=1
    return(n)

#this function takes as argument the return of list_pairs and the minimum energy needed to turn on the pixel
#it returns the number of particles with enough energy to light a pixel

pixel = compte_pixel(sample_after_scattering,Wi)
print("number total of pixel:", len(samples),"number of lit pixel", pixel)

number total of pixel: 9999 number of lit pixel 9999
